In [ ]:
from azure.quantum.qiskit import AzureQuantumProvider
from qiskit import QuantumCircuit, transpile
#from qiskit.circuit.library import RGQFTMultiplier
#from qiskit.tools.monitor import job_monitor

from qiskit.circuit.library import ZZFeatureMap
from qiskit.quantum_info import DensityMatrix,partial_trace

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
df_results = pd.DataFrame(columns = ['n', 'error_budget', 'physical_qubits', 'physical_qubits_STD', 'runtime', 'runtime_STD'])

n_range = np.array([  2,   3,   4,   5,   6,   7,   8,   9,  10,  15,  20,  30,  40,
        50,  60,  70,  80,  90, 100])

error_budgets = np.array([0.49172223, 0.37372519, 0.30123701, 0.25212096, 0.21659496,
       0.18966802, 0.16852711, 0.15146606, 0.13738998, 0.09243216,
       0.06791182, 0.0412309 , 0.02679642, 0.01789353, 0.01208699,
       0.00820593, 0.00558348, 0.00380287, 0.00259126])

plt.plot(n_range,error_budgets)
plt.yscale('log')
plt.show()

In [ ]:
i = 0
QRE_repetitions = 1

n = n_range[i]
error = error_budgets[i]
X = np.random.random((QRE_repetitions, n))

print(n)

In [ ]:
def ZZ_map_tomography(X, backend, info=False):
    N_FEATURES = X.shape[0]
    ZZ_reps = 5
    ent_type = 'full'
    qc = QuantumCircuit(N_FEATURES)
    fm = ZZFeatureMap(feature_dimension=N_FEATURES, reps=ZZ_reps, entanglement=ent_type)
    fm_bound = fm.assign_parameters(X)
    qc.append(fm_bound, range(N_FEATURES))
    qc.measure_all()

    if(info): print(f"[INFO] Decompose circuit into intrinsic quantum operations")

    # retrieve basis gates from backend
    basis_gates = backend.configuration().basis_gates
    circ = transpile(qc, basis_gates=basis_gates, optimization_level=0)

    # print some statistics
    if(info): print(f"[INFO]   qubit count: {circ.num_qubits}")
    if(info): print("[INFO]   gate counts")
    for gate, count in circ.count_ops().items():
        if(info): print(f"[INFO]   - {gate}: {count}")

    return(qc)

In [ ]:
from azure.quantum import Workspace
from azure.quantum.qiskit import AzureQuantumProvider

workspace = Workspace(
            resource_id = #resource id,
            location = #location)
)


provider = AzureQuantumProvider(workspace)

backend = provider.get_backend('microsoft.estimator')

In [ ]:
l_results = []

for x in X:
    print(x)
    circ = ZZ_map_tomography(x, backend)
    job = backend.run(circ,
        errorBudget=float(error))
    #job_monitor(job)
    result = job.result()
    l_results.append(result)


In [ ]:
result

In [ ]:
physical_qubits = np.empty((len(l_results),))
runtimes = np.empty((len(l_results),))
for i, result in enumerate(l_results):
    physical_qubits[i] = result.data()["physicalCounts"]['physicalQubits']
    runtimes[i] = result.data()["physicalCounts"]['runtime']


In [ ]:
print("Physical Qubits: [mean] ", physical_qubits.mean(), ", [std] ", physical_qubits.std())
print("Runtime: [mean] ", runtimes.mean(), ", [std] ", runtimes.std())

In [ ]:
df_results.loc[len(df_results.index)] = [n, error, physical_qubits.mean(), physical_qubits.std(), runtimes.mean(), runtimes.std()]  

In [ ]:
df_results